# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
data = "cities_data.csv"
weather_data = pd.read_csv(data)
weather_data = weather_data.dropna()
del weather_data["Unnamed: 0"]
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Hualmay,-11.0964,-77.6139,76.21,64,99,8.95
1,Lorengau,-2.0226,147.2712,81.90,78,100,6.76
2,Cape Town,-33.9258,18.4232,66.25,63,0,18.41
3,Rio Claro,-22.4114,-47.5614,86.00,39,20,11.50
4,Dikson,73.5069,80.5464,-32.01,85,90,14.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
gmaps.configure(api_key=g_key)

In [63]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

In [64]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
dissipating=False, max_intensity=100,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
vacation_cloud = weather_data[weather_data["Cloudiness"] <5]
vacation_min_temp = vacation_cloud[vacation_cloud["Max Temp"] > 65]
vacation_max_temp = vacation_min_temp[vacation_min_temp["Max Temp"] < 90]
vacation_wind = vacation_max_temp[vacation_max_temp["Wind Speed"] < 5]
vacation_humidity = vacation_wind[vacation_wind["Humidity"] < 70]

In [46]:
perfect_vacation = vacation_humidity.reset_index()
perfect_vacation.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,12,Hilo,19.7297,-155.0900,74.88,53,1,4.61
1,36,Guerrero Negro,27.9769,-114.0611,76.14,19,0,4.52
2,37,Luján,-34.5703,-59.1050,87.66,43,0,3.00
3,47,Kapaa,22.0752,-159.3190,71.26,56,1,2.01
4,60,Castro,-24.7911,-50.0119,75.11,54,0,3.20


In [47]:
del perfect_vacation["index"]

In [48]:
perfect_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Hilo,19.7297,-155.0900,74.88,53,1,4.61
1,Guerrero Negro,27.9769,-114.0611,76.14,19,0,4.52
2,Luján,-34.5703,-59.1050,87.66,43,0,3.00
3,Kapaa,22.0752,-159.3190,71.26,56,1,2.01
4,Castro,-24.7911,-50.0119,75.11,54,0,3.20
5,Jardim,-21.4803,-56.1381,86.14,38,0,4.97
6,Kédougou,12.5579,-12.1743,82.69,15,0,4.52
7,Campana,-34.1687,-58.9591,87.67,45,0,4.00
8,Māngrol,21.1167,70.1167,69.69,50,0,2.89
9,Caborca,30.5838,-111.1010,73.00,27,0,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
hotels = []


for city in range(len(indexed_perfect_vacation["City"])):
    lat = perfect_vacation.loc[city]["Lat"]
    lng = perfect_vacation.loc[city]["Lng"]
    city_loc = f"{lat},{lng}"
    params = {"location": city_loc, 
            "types": "hotels",
            "radius": 5000,
            "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("no hotels found")
perfect_vacation["Nearest Hotel"] = hotels
perfect_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Nearest Hotel
0,Hilo,19.7297,-155.0900,74.88,53,1,4.61,Hilo
1,Guerrero Negro,27.9769,-114.0611,76.14,19,0,4.52,Guerrero Negro
2,Luján,-34.5703,-59.1050,87.66,43,0,3.00,Luján
3,Kapaa,22.0752,-159.3190,71.26,56,1,2.01,Kapaʻa
4,Castro,-24.7911,-50.0119,75.11,54,0,3.20,Castro
5,Jardim,-21.4803,-56.1381,86.14,38,0,4.97,Jardim
6,Kédougou,12.5579,-12.1743,82.69,15,0,4.52,Kedougou
7,Campana,-34.1687,-58.9591,87.67,45,0,4.00,Campana
8,Māngrol,21.1167,70.1167,69.69,50,0,2.89,Mangrol
9,Caborca,30.5838,-111.1010,73.00,27,0,1.99,Santa Ana


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_vacation.iterrows()]
locations = perfect_vacation[["Lat", "Lng"]]

In [67]:
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))